# UZOP Projekt ; Predicting NBA Shots

## Studenti:
- Nikola Kraljevc
- Ana Ujevic
- Timon Rodic


Rok za predaju prvog dijela projekta je 10.11.2023. 23:59

Prvi dio projekta nosi 10 bodova.

Za prvi rok predaje trebate: 
- dobro proučiti cijeli članak
- upoznati se s podatcima
- napraviti čiščenje i vizualizaciju podataka
- Bitno mi je da provedete detaljnu deskriptivnu statistiku
- da se upoznate s nedostajećim vrijednostima i da razmislite na koji bi način riješili problem s pojedinim nedostajećim vrijednostima
- da napravite vizualizacije na način na koji smatrate da je najbolje uočiti informacije koje možemo izvući iz tih podataka.

Dok to budete radili možete se voditi člankom, ali i ne morate. Bitno je samo da se dobro upoznate sa svakim dijelom dataseta. Za sve korake implementacije pišite komentare u jupyter notebook. 